In [1]:
import IJulia

# The julia kernel has built in support for Revise.jl, so this is the 
# recommended approach for long-running sessions:
# https://github.com/JuliaLang/IJulia.jl/blob/9b10fa9b879574bbf720f5285029e07758e50a5e/src/kernel.jl#L46-L51

# Users should enable revise within .julia/config/startup_ijulia.jl:
# https://timholy.github.io/Revise.jl/stable/config/#Using-Revise-automatically-within-Jupyter/IJulia-1

# clear console history
IJulia.clear_history()

fig_width = 7
fig_height = 5
fig_format = :retina
fig_dpi = 96

# no retina format type, use svg for high quality type/marks
if fig_format == :retina
  fig_format = :svg
elseif fig_format == :pdf
  fig_dpi = 96
  # Enable PDF support for IJulia
  IJulia.register_mime(MIME("application/pdf"))
end

# convert inches to pixels
fig_width = fig_width * fig_dpi
fig_height = fig_height * fig_dpi

# Intialize Plots w/ default fig width/height
try
  import Plots

  # Plots.jl doesn't support PDF output for versions < 1.28.1
  # so use png (if the DPI remains the default of 300 then set to 96)
  if (Plots._current_plots_version < v"1.28.1") & (fig_format == :pdf)
    Plots.gr(size=(fig_width, fig_height), fmt = :png, dpi = fig_dpi)
  else
    Plots.gr(size=(fig_width, fig_height), fmt = fig_format, dpi = fig_dpi)
  end
catch e
  # @warn "Plots init" exception=(e, catch_backtrace())
end

# Initialize CairoMakie with default fig width/height
try
  import CairoMakie
  
  CairoMakie.activate!(type = string(fig_format))
  CairoMakie.update_theme!(resolution=(fig_width, fig_height))
catch e
    # @warn "CairoMakie init" exception=(e, catch_backtrace())
end
  
# Set run_path if specified
try
  run_path = raw"C:\Users\alex\Documents\repos\exess\smolkaa.github.io\api\grids"
  if !isempty(run_path)
    cd(run_path)
  end
catch e
  @warn "Run path init:" exception=(e, catch_backtrace())
end


# emulate old Pkg.installed beahvior, see
# https://discourse.julialang.org/t/how-to-use-pkg-dependencies-instead-of-pkg-installed/36416/9
import Pkg
function isinstalled(pkg::String)
  any(x -> x.name == pkg && x.is_direct_dep, values(Pkg.dependencies()))
end

# ojs_define
if isinstalled("JSON") && isinstalled("DataFrames")
  import JSON, DataFrames
  global function ojs_define(; kwargs...)
    convert(x) = x
    convert(x::DataFrames.AbstractDataFrame) = Tables.rows(x)
    content = Dict("contents" => [Dict("name" => k, "value" => convert(v)) for (k, v) in kwargs])
    tag = "<script type='ojs-define'>$(JSON.json(content))</script>"
    IJulia.display(MIME("text/html"), tag)
  end
elseif isinstalled("JSON")
  import JSON
  global function ojs_define(; kwargs...)
    content = Dict("contents" => [Dict("name" => k, "value" => v) for (k, v) in kwargs])
    tag = "<script type='ojs-define'>$(JSON.json(content))</script>"
    IJulia.display(MIME("text/html"), tag)
  end
else
  global function ojs_define(; kwargs...)
    @warn "JSON package not available. Please install the JSON.jl package to use ojs_define."
  end
end


# don't return kernel dependencies (b/c Revise should take care of dependencies)
nothing


In [2]:
#| echo: false
if !isdefined(Main, :ExESS)
include(joinpath(@__DIR__, "..", "..", "..", "exess.jl", "src", "ExESS.jl"))
using .ExESS
end
@doc ExESS.GlobalStructured3DGrid

```
[1] struct GlobalStructured3DGrid{T<:Real} <: AbstractGlobalStructured3DGrid
[2] GlobalStructured3DGrid([T::Type,] r0::Real, h::AbstractVector, N_theta::Int64, N_phi::Int64)
[3] GlobalStructured3DGrid([T::Type,] r::AbstractVector, N_theta::Int64, N_phi::Int64)
```

Global structured volume grid (3D) of type `GlobalSphericalPosition{T}` over a sphere of radius `r0` with heights of the individual radial layers `h`. Alternatively, the radial  distances can directly be specified by `r` (`r = r0 .+ h`). Note that all heights have to be positive.

| Field     | Type                                 | Description                               |
|:--------- |:------------------------------------ |:----------------------------------------- |
| `r0`      | `T`                                  | radius of global body (sphere)            |
| `h`       | `Vector{T}`                          | heights above radial base `r0`            |
| `N_r`     | `Int64`                              | number of elements in radial direction    |
| `N_theta` | `Int64`                              | number of elements in azimuth direction   |
| `N_phi`   | `Int64`                              | number of elements in elevation direction |
| `coords`  | `Vector{GlobalSphericalPosition{T}}` | coordinates                               |
| `areas`   | `Vector{T}`                          | surface area                              |
| `volumes` | `Vector{T}`                          | volumes                                   |

To ensure expected behavior, the grid object should generally be created with the outer constructors [2] or [3].


In [3]:
#| echo: false
if !isdefined(Main, :ExESS)
include(joinpath(@__DIR__, "..", "..", "..", "exess.jl", "src", "ExESS.jl"))
using .ExESS
end
@doc ExESS.GlobalStructured3DGrid_EqSim

```
[1] struct GlobalStructured3DGrid_EqSim{T<:Real} <: AbstractGlobalStructured3DGrid end
[2] GlobalStructured3DGrid_EqSim([T::Type,] r0::Real, h::AbstractVector, N_theta::Int64, N_phi::Int64)
[3] GlobalStructured3DGrid_EqSim([T::Type,] r::AbstractVector, N_theta::Int64, N_phi::Int64)
```

Global structured volume grid (3D) of type `GlobalSphericalPosition{T}` over a hemisphere of radius `r0` with heights of the individual radial layers `h`. Alternatively, the radial  distances can directly be specified by `r` (`r = r0 .+ h`). Note that all heights have to be positive.

| Field     | Type                                 | Description                               |
|:--------- |:------------------------------------ |:----------------------------------------- |
| `r0`      | `T`                                  | radius of global body (sphere)            |
| `h`       | `Vector{T}`                          | heights above radial base `r0`            |
| `N_r`     | `Int64`                              | number of elements in radial direction    |
| `N_theta` | `Int64`                              | number of elements in azimuth direction   |
| `N_phi`   | `Int64`                              | number of elements in elevation direction |
| `coords`  | `Vector{GlobalSphericalPosition{T}}` | coordinates                               |
| `areas`   | `Vector{T}`                          | surface area                              |
| `volumes` | `Vector{T}`                          | volumes                                   |

To ensure expected behavior, the grid object should generally be created with the outer constructors [2] or [3].


In [4]:
#| echo: false
if !isdefined(Main, :ExESS)
include(joinpath(@__DIR__, "..", "..", "..", "exess.jl", "src", "ExESS.jl"))
using .ExESS
end
@doc ExESS.GlobalStructured3DGrid_Reduced

```
[1] struct GlobalStructured3DGrid_Reduced{T<:Real} <: AbstractGlobalStructured3DGrid end
[2] GlobalStructured3DGrid_Reduced([T::Type,] r0::Real, h::AbstractVector, N_phi::Int64)
[3] GlobalStructured3DGrid_Reduced([T::Type,] r::AbstractVector, N_phi::Int64)
```

Global structured volume grid (3D) of type `GlobalSphericalPosition{T}` over a sphere of radius `r0` with heights of the individual radial layers `h`. Alternatively, the radial  distances can directly be specified by `r` (`r = r0 .+ h`). Note that all heights have to be positive. The grid is reduced in the azimuth direction to have approximately equal  `2*pi*r*cos(phi)/N_theta` grid element lengths.

| Field     | Type                                 | Description                               |
|:--------- |:------------------------------------ |:----------------------------------------- |
| `r0`      | `T`                                  | radius of global body (sphere)            |
| `h`       | `Vector{T}`                          | heights above radial base `r0`            |
| `N_r`     | `Int64`                              | number of elements in radial direction    |
| `N_theta` | `Int64`                              | number of elements in azimuth direction   |
| `N_phi`   | `Int64`                              | number of elements in elevation direction |
| `coords`  | `Vector{GlobalSphericalPosition{T}}` | coordinates                               |
| `areas`   | `Vector{T}`                          | surface area                              |
| `volumes` | `Vector{T}`                          | volumes                                   |

To ensure expected behavior, the grid object should generally be created with the outer constructors [2] or [3].


In [5]:
#| echo: false
if !isdefined(Main, :ExESS)
include(joinpath(@__DIR__, "..", "..", "..", "exess.jl", "src", "ExESS.jl"))
using .ExESS
end
@doc ExESS.GlobalStructured3DGrid_Reduced_EqSim

```
[1] struct GlobalStructured3DGrid_Reduced_EqSim{T<:Real} <: AbstractGlobalStructured3DGrid end
[2] GlobalStructured3DGrid_Reduced_EqSim([T::Type,] r0::Real, h::AbstractVector, N_phi::Int64)
[3] GlobalStructured3DGrid_Reduced_EqSim([T::Type,] r::AbstractVector, N_phi::Int64)
```

Global structured volume grid (3D) of type `GlobalSphericalPosition{T}` over a sphere of radius `r0` with heights of the individual radial layers `h`. Alternatively, the radial  distances can directly be specified by `r` (`r = r0 .+ h`). Note that all heights have to be positive. The grid is reduced in the azimuth direction to have approximately equal  `2*pi*r*cos(phi)/N_theta` grid element lengths.

| Field     | Type                                 | Description                               |
|:--------- |:------------------------------------ |:----------------------------------------- |
| `r0`      | `T`                                  | radius of global body (sphere)            |
| `h`       | `Vector{T}`                          | heights above radial base `r0`            |
| `N_r`     | `Int64`                              | number of elements in radial direction    |
| `N_theta` | `Int64`                              | number of elements in azimuth direction   |
| `N_phi`   | `Int64`                              | number of elements in elevation direction |
| `coords`  | `Vector{GlobalSphericalPosition{T}}` | coordinates                               |
| `areas`   | `Vector{T}`                          | surface area                              |
| `volumes` | `Vector{T}`                          | volumes                                   |

To ensure expected behavior, the grid object should generally be created with the outer constructors [2] or [3].
